<a href="https://colab.research.google.com/github/SakshiGehani/J015_NLPassignments/blob/ClassWork/CW5_Word2Vec_J015_19_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

In [2]:
amazon = pd.read_json("http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Office_Products_5.json.gz", lines=True)
amazon.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A32T2H8150OJLU,B00000JBLH,ARH,"[3, 4]","I bought my first HP12C in about 1984 or so, a...",5,"A solid performer, and long time friend",1094169600,"09 3, 2004"
1,A3MAFS04ZABRGO,B00000JBLH,"Let it Be ""Alan""","[7, 9]",WHY THIS BELATED REVIEW? I feel very obliged t...,5,"Price of GOLD is up, so don't bury the golden ...",1197676800,"12 15, 2007"
2,A1F1A0QQP2XVH5,B00000JBLH,Mark B,"[3, 3]",I have an HP 48GX that has been kicking for mo...,2,"Good functionality, but not durable like old HPs",1293840000,"01 1, 2011"
3,A49R5DBXXQDE5,B00000JBLH,R. D Johnson,"[7, 8]",I've started doing more finance stuff recently...,5,One of the last of an almost extinct species,1145404800,"04 19, 2006"
4,A2XRMQA6PJ5ZJ8,B00000JBLH,Roger J. Buffington,"[0, 0]",For simple calculations and discounted cash fl...,5,Still the best,1375574400,"08 4, 2013"


In [0]:
import gensim.downloader as api

In [0]:
corpus = api.load('text8')

In [5]:
!pip install Word2Vec

In [6]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec

path = get_tmpfile("word2vec.model")
model = Word2Vec(common_texts, size=100, window=5, min_count=1, workers=4, iter=10)
model.save("word2vec.model")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


RevText clm-> remove stop words(punc,articles,conjunctions)-> remove duplicates->list of unique words->combination(itertools)[with repetition] i.e pairs of 2 -->gives tuple ;convert to dataframe  -> wv.similarity (pass word1,word2 of combinations)-> gives similarity score

In [7]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

#example_sent = "This is a sample sentence, showing off the stop words filtration."
sentence = amazon.reviewText[0]     ## take the first row/string of reviewText column

stop_words = set(stopwords.words('english'))

word_tokens = word_tokenize(sentence.lower())     # convert string to list of words

filtered_sentence = [w for w in word_tokens if not w in stop_words]     # eliminate stop words

'''filtered_sentence = []
for w in word_tokens:
    if w not in stop_words:
        filtered_sentence.append(w)'''

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'filtered_sentence = []\nfor w in word_tokens:\n    if w not in stop_words:\n        filtered_sentence.append(w)'

In [8]:
print(filtered_sentence)

['bought', 'first', 'hp12c', '1984', ',', 'served', 'faithfully', '2002', 'lost', 'travelling', '.', 'searched', 'another', 'one', 'replace', ',', 'found', 'one', 'difficult', 'come', 'area', '.', ',', 'decided', 'buy', 'purchased', 'hp', '49g', '.', 'mistake', '!', 'know', 'many', 'people', 'view', 'hp', '49g', '(', '49g+', ')', 'flagship', 'hp', 'line', ',', 'disaster.the', '49g', 'may', 'powerful', ',', 'using', 'extremely', 'counterintuitive', '...', 'manual', 'sketchy', 'best', '.', '12c', ',', 'hand', ',', 'need', 'way', 'makes', 'good', 'sense', 'me.if', 'looking', 'solid', ',', 'reliable', 'calculator', ',', 'hp12c', 'may', '.', "'s", 'programmable', '.', 'basic', 'statistics', 'well', ',', 'many', 'business', 'applications', '.', 'manual', 'makes', 'sense', ';', 'running', 'short', 'order.i', "'m", 'ready', 'set', '49g', 'aside', 'move', 'back', 'old', 'friend', '.', "n't", 'even', 'replace', 'batteries', 'well', 'decade', 'use', '!', 'hp', '12c', ',', "'m", 'coming', 'home', 

In [9]:
import re
def remove_digits(list): 
    pattern = '[0-9]'
    list = [re.sub(pattern, '', i) for i in list]           ## remove digits from string
    return list

filtered_sentence = remove_digits(filtered_sentence)
print(filtered_sentence)

['bought', 'first', 'hpc', '', ',', 'served', 'faithfully', '', 'lost', 'travelling', '.', 'searched', 'another', 'one', 'replace', ',', 'found', 'one', 'difficult', 'come', 'area', '.', ',', 'decided', 'buy', 'purchased', 'hp', 'g', '.', 'mistake', '!', 'know', 'many', 'people', 'view', 'hp', 'g', '(', 'g+', ')', 'flagship', 'hp', 'line', ',', 'disaster.the', 'g', 'may', 'powerful', ',', 'using', 'extremely', 'counterintuitive', '...', 'manual', 'sketchy', 'best', '.', 'c', ',', 'hand', ',', 'need', 'way', 'makes', 'good', 'sense', 'me.if', 'looking', 'solid', ',', 'reliable', 'calculator', ',', 'hpc', 'may', '.', "'s", 'programmable', '.', 'basic', 'statistics', 'well', ',', 'many', 'business', 'applications', '.', 'manual', 'makes', 'sense', ';', 'running', 'short', 'order.i', "'m", 'ready', 'set', 'g', 'aside', 'move', 'back', 'old', 'friend', '.', "n't", 'even', 'replace', 'batteries', 'well', 'decade', 'use', '!', 'hp', 'c', ',', "'m", 'coming', 'home', '!']


In [10]:
punctuations = ['',',', '.', ':', ';', '-', '?', '!', '"', "'", '[', ']', '{', '}', '|', '/', '<', '>']
new_filtered = [w for w in filtered_sentence if not w in punctuations]      # eliminate white spaces and punctuation marks
print(new_filtered)

['bought', 'first', 'hpc', 'served', 'faithfully', 'lost', 'travelling', 'searched', 'another', 'one', 'replace', 'found', 'one', 'difficult', 'come', 'area', 'decided', 'buy', 'purchased', 'hp', 'g', 'mistake', 'know', 'many', 'people', 'view', 'hp', 'g', '(', 'g+', ')', 'flagship', 'hp', 'line', 'disaster.the', 'g', 'may', 'powerful', 'using', 'extremely', 'counterintuitive', '...', 'manual', 'sketchy', 'best', 'c', 'hand', 'need', 'way', 'makes', 'good', 'sense', 'me.if', 'looking', 'solid', 'reliable', 'calculator', 'hpc', 'may', "'s", 'programmable', 'basic', 'statistics', 'well', 'many', 'business', 'applications', 'manual', 'makes', 'sense', 'running', 'short', 'order.i', "'m", 'ready', 'set', 'g', 'aside', 'move', 'back', 'old', 'friend', "n't", 'even', 'replace', 'batteries', 'well', 'decade', 'use', 'hp', 'c', "'m", 'coming', 'home']


In [11]:
def remove_dupes(x):
  return list(dict.fromkeys(x))               ## remove duplicate words by coverting words in the list to keys in a dictionary and then again converting to list
                                    # This automatically removes any duplicate words

unique = remove_dupes(new_filtered)
print(unique)

['bought', 'first', 'hpc', 'served', 'faithfully', 'lost', 'travelling', 'searched', 'another', 'one', 'replace', 'found', 'difficult', 'come', 'area', 'decided', 'buy', 'purchased', 'hp', 'g', 'mistake', 'know', 'many', 'people', 'view', '(', 'g+', ')', 'flagship', 'line', 'disaster.the', 'may', 'powerful', 'using', 'extremely', 'counterintuitive', '...', 'manual', 'sketchy', 'best', 'c', 'hand', 'need', 'way', 'makes', 'good', 'sense', 'me.if', 'looking', 'solid', 'reliable', 'calculator', "'s", 'programmable', 'basic', 'statistics', 'well', 'business', 'applications', 'running', 'short', 'order.i', "'m", 'ready', 'set', 'aside', 'move', 'back', 'old', 'friend', "n't", 'even', 'batteries', 'decade', 'use', 'coming', 'home']


In [0]:
# Function which returns subsets/combinations of r length from n elements
from itertools import combinations      ## w/o replacement
  
def rSubset(arr, r): 
  
    # return list of all subsets of length r 
    # to deal with duplicate subsets use  
    set(list(combinations(arr, r))) 
    return list(combinations(arr, r)) 

In [13]:
pairs = rSubset(unique, 2)        ## creating pairs/combinations of 2 words each
print(pairs)

[('bought', 'first'), ('bought', 'hpc'), ('bought', 'served'), ('bought', 'faithfully'), ('bought', 'lost'), ('bought', 'travelling'), ('bought', 'searched'), ('bought', 'another'), ('bought', 'one'), ('bought', 'replace'), ('bought', 'found'), ('bought', 'difficult'), ('bought', 'come'), ('bought', 'area'), ('bought', 'decided'), ('bought', 'buy'), ('bought', 'purchased'), ('bought', 'hp'), ('bought', 'g'), ('bought', 'mistake'), ('bought', 'know'), ('bought', 'many'), ('bought', 'people'), ('bought', 'view'), ('bought', '('), ('bought', 'g+'), ('bought', ')'), ('bought', 'flagship'), ('bought', 'line'), ('bought', 'disaster.the'), ('bought', 'may'), ('bought', 'powerful'), ('bought', 'using'), ('bought', 'extremely'), ('bought', 'counterintuitive'), ('bought', '...'), ('bought', 'manual'), ('bought', 'sketchy'), ('bought', 'best'), ('bought', 'c'), ('bought', 'hand'), ('bought', 'need'), ('bought', 'way'), ('bought', 'makes'), ('bought', 'good'), ('bought', 'sense'), ('bought', 'me.i

In [14]:
pairs_df = pd.DataFrame(pairs, columns=['Word1', 'Word2'])      ## storing the combinations/tuples as dataframe
pairs_df.head() 

,Word1,Word2
0,bought,first
1,bought,hpc
2,bought,served
3,bought,faithfully
4,bought,lost


In [0]:
#@title
'''def sim_score(w1, w2):
  return model.similarity(w1, w2)

import numpy as np
def cos_sim(w1, w2):
  cosine_similarity = np.dot(model[w1], model[w2])/(np.linalg.norm(model[w1])* numpy.linalg.norm(model[w2]))
  return cosine_similarity'''

In [0]:
#@title
'''pairs_df['Similarity Score'] = np.vectorize(cos_sim)(pairs_df['Word1'], pairs_df['Word2'])'''

In [15]:
bagofwords = [[i] for i in unique]
model1 = Word2Vec(list(bagofwords), min_count=1,size= 50,workers=3, window =3, sg = 1)
print(bagofwords)

[['bought'], ['first'], ['hpc'], ['served'], ['faithfully'], ['lost'], ['travelling'], ['searched'], ['another'], ['one'], ['replace'], ['found'], ['difficult'], ['come'], ['area'], ['decided'], ['buy'], ['purchased'], ['hp'], ['g'], ['mistake'], ['know'], ['many'], ['people'], ['view'], ['('], ['g+'], [')'], ['flagship'], ['line'], ['disaster.the'], ['may'], ['powerful'], ['using'], ['extremely'], ['counterintuitive'], ['...'], ['manual'], ['sketchy'], ['best'], ['c'], ['hand'], ['need'], ['way'], ['makes'], ['good'], ['sense'], ['me.if'], ['looking'], ['solid'], ['reliable'], ['calculator'], ["'s"], ['programmable'], ['basic'], ['statistics'], ['well'], ['business'], ['applications'], ['running'], ['short'], ['order.i'], ["'m"], ['ready'], ['set'], ['aside'], ['move'], ['back'], ['old'], ['friend'], ["n't"], ['even'], ['batteries'], ['decade'], ['use'], ['coming'], ['home']]


In [0]:
vocabulary = model1.wv.vocab
def sim_score(w1, w2):
  return model1.wv.similarity(w1, w2)             # calculating similarity b/w 2 words

In [17]:
import numpy as np
pairs_df['Similarity Score'] = np.vectorize(sim_score)(pairs_df['Word1'], pairs_df['Word2'])
# finding similarity score for each combination

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [18]:
pairs_df.head()

,Word1,Word2,Similarity Score
0,bought,first,0.009798
1,bought,hpc,-0.197298
2,bought,served,-0.096991
3,bought,faithfully,0.019584
4,bought,lost,0.051421
